In [165]:
from tqdm.notebook import tqdm

In [135]:
def parse_input(fl):
    data = []
    with open(fl) as infile:
        for ln in infile.readlines():
            row, _cnts = ln.strip().split()
            cnts = [int(c) for c in _cnts.split(',')]
            data.append({'row': row, 'cnts': cnts})
    return data

In [136]:
test = parse_input("data/day12-test.txt")
inputs = parse_input("data/day12-input.txt")
test

[{'row': '???.###', 'cnts': [1, 1, 3]},
 {'row': '.??..??...?##.', 'cnts': [1, 1, 3]},
 {'row': '?#?#?#?#?#?#?#?', 'cnts': [1, 3, 1, 6]},
 {'row': '????.#...#...', 'cnts': [4, 1, 1]},
 {'row': '????.######..#####.', 'cnts': [1, 6, 5]},
 {'row': '?###????????', 'cnts': [3, 2, 1]}]

## Part 1

In [168]:
'?'.join(['adada']*5)

'adada?adada?adada?adada?adada'

In [169]:
def part1(data):
    tot = 0
    # print("cnt\trow")
    for r in data:
        paths = []
        row_cnt = num_ways(
            row=r['row'],
            cnts=r['cnts'],
            path=list(r['row']),
            paths=paths,
        )
        # print(f"{row_cnt}\t{r}")
        tot += row_cnt
    return tot

def part2(data):
    tot = 0
    for r in tqdm(data, desc='Rows'):
        tot += num_ways(
            row='?'.join([r['row']]*5),
            cnts=r['cnts']*5,
        )
    return tot
    
def num_ways(row, cnts, path=None, paths=None):
    row_sz, num_cnt = len(row), len(cnts)
    cache = [[None]*num_cnt for _ in range(row_sz)]
    return num_ways_rec(rix=0, cix=0, row=row, cnts=cnts, cache=cache,path=path, paths=paths)


def num_ways_rec(rix, cix, row, cnts, cache, path=None, paths=None):
    # Terminal condition
    row_sz = len(row)
    if (
        cix == len(cnts)
        and 
        (
            # The end of row as well
            (rix >= row_sz)
            # Only working gears
            or (set(row[rix:]) <= {'.', '?'})
        )
    ):
        if path:
            joint_path = ''.join(path[:])
            assert sum(c=='#' for c in path) == sum(cnts), f"{row=}, {cnts=}\n{joint_path=}"
            paths.append(joint_path)
        return 1
    elif cix == len(cnts):
        return 0
    elif rix >= row_sz:
        return 0
    # Logic
    if cache[rix][cix] is not None:
        return cache[rix][cix]
    for ri in range(rix, row_sz):
        if row[ri] in {'?', '#'}:
            break
    else:
        return 0
    if cache[ri][cix] is not None:
        return cache[ri][cix]
    cur_cnt = cnts[cix]
    kwargs = dict(
        row=row,
        cnts=cnts,
        cache=cache,
        path=path,
        paths=paths,
    )
    if (
        (ri==0 or row[ri-1]!='#')
        and ri+cur_cnt <= row_sz
        and (set(row[ri:ri+cur_cnt]) <= {'?', '#'})
        and (ri+cur_cnt >= row_sz or row[ri+cur_cnt] != '#')
    ):
        if path:
            orig_val = path[ri:ri+cur_cnt]
            path[ri:ri+cur_cnt] =  ['#'] * cur_cnt
        filled_cnt = num_ways_rec(
            # plus one as we cannot start another
            # damaged contiguous group immediately after
            # a filled contiguous group
            rix=ri+cur_cnt+1,
            cix=cix+1,
            **kwargs,
        )
        if path:
            path[ri:ri+cur_cnt] = orig_val
    else:
        filled_cnt = 0
    if row[ri] == '?':
        # we can only choose to fill if qn mark
        non_filled_cnt = num_ways_rec(
            rix=ri+1,
            cix=cix,
            **kwargs,
        )
    else:
        non_filled_cnt = 0
    tot_cnt = filled_cnt + non_filled_cnt
    cache[rix][cix] = cache[ri][cix] = tot_cnt
    # print(f"{ri}\t{cix}\t{cur_cnt}\t{row[ri:ri+cur_cnt]}\t{filled_cnt}\t\t{non_filled_cnt}\t\t{tot_cnt}")
    return tot_cnt

In [163]:
part1(data=test)

21

In [164]:
part1(data=inputs)

7084

In [170]:
part2(data=test)

Rows:   0%|          | 0/6 [00:00<?, ?it/s]

525152

In [171]:
part2(data=inputs)

Rows:   0%|          | 0/1000 [00:00<?, ?it/s]

8414003326821